In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

c:\Users\hp\Desktop\NLP-Project\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import faiss 
import numpy as np
import pickle

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
class semanticEmbedding:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    def get_embedding(self, sentences):
    # Tokenize sentences
        encoded_input = self.tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**encoded_input)
        # Perform pooling
        sentence_embeddings = self.mean_pooling(model_output, encoded_input['attention_mask'])

        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings.detach().numpy()

In [5]:
class FaissIdx:

    def __init__(self, model, dim=768):
        self.index = faiss.IndexFlatIP(dim) # need to load the pickle model in the final file
        # self.index = faiss
        # Maintaining the document data
        self.doc_map = dict()
        self.model = model
        self.ctr = 0

    def add_doc(self, document_text):
        self.index.add(self.model.get_embedding(document_text))
        self.doc_map[self.ctr] = document_text # store the original document text
        self.ctr += 1

    def search_doc(self, query, k=3):
        D, I = self.index.search(self.model.get_embedding(query), 5)
        return [{self.doc_map[idx]: score} for idx, score in zip(I[0], D[0]) if idx in self.doc_map]
    def save_index(self, index_filename, doc_map_filename):
        # Save Faiss index to file
        faiss.write_index(self.index, index_filename)

        # Save document map to file using pickle
        with open(doc_map_filename, 'wb') as f:
            pickle.dump(self.doc_map, f)

    def load_index(self, index_filename, doc_map_filename):
        # Load Faiss index from file
        self.index = faiss.read_index(index_filename)

        # Load document map from file using pickle
        with open(doc_map_filename, 'rb') as f:
            self.doc_map = pickle.load(f)

In [6]:
# creating an instance of the class
model = semanticEmbedding()

In [7]:
index = FaissIdx(model)

In [8]:
index.load_index('index.bin', 'doc_map.pkl')

In [9]:
import google.generativeai as genai

In [10]:
model = genai.GenerativeModel('gemini-pro')

In [11]:
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))

# given the query, run the following code sequentially

In [21]:
query = "Iam kishan from uttarpradesh, and I want subsidy"

In [23]:
user_name = 'Kishan'

In [24]:
website = 'https://www.india.gov.in/my-government/schemes'

In [25]:
query_result = index.search_doc(query)

In [26]:
context = ''

In [31]:
for i in range(5):
    for key, value in query_result[i].items():
        if(value>0.23):
            context += key + " "

In [32]:
context

"Supply of Food Grains to Welfare Institutes, Chhattisgarh = Under this scheme, each beneficiary of the welfare institutes of the state is eligible to get 15 Kg food grains every month at BPL rate.Village Grain Bank Scheme by Department of Food and Public Distribution = Get information about the Village Grain Bank Scheme provided by the Department of Food and Public Distribution. The objective of the Scheme is to provide safeguard against starvation during the period of natural calamity or during lean season when the marginalized food insecure households do not have sufficient resources to purchase rations. Users can get details about the eligibility criteria, beneficiary type and benefits of the Scheme. Information on how to avail the Scheme is also provided.Mukhamantri Khadyan Sahayta Yojna, Chhattisgarh = Under this scheme, 35 Kg rice@ Rs.1/kg to 7.66 Lacs State AAY families; 10 kg rice free of cost, to 60,277 destitute ration card holders and 10 kg rice @ rs. 1/kg to 8,194 disabled

In [34]:
content = ""

In [35]:
if context == '':
    print('Is there anything else I can help you with?')
else:
    formatted_response = f''' Give response in this format, dont change the format or the links given-: 
Hello {user_name},

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. Scheme Name 1:
   - Information like eligibility, benefit.

2. Scheme Name 2:
   - Information like eligibility, benefit.

3. Scheme Name 3:
   - Information like eligibility, benefit.

4. Scheme Name 4:
   - Information like eligibility, benefit.

5. Scheme Name 5:
   - Information like eligibility, benefit.


For more detailed information or to apply, you can visit the official [Government Department/Agency] website: {website}.


If you have any more questions or need further assistance, feel free to ask!

Best regards,
GovSevak.

*This message is generated by GENAI, GENAI can make mistakes. Consider checking important information.*

QUERY:${query}$ CONTEXT:${context}$'''

    response = model.generate_content(formatted_response)
    print(response.text)



Hello Kishan,

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. Mukhamantri Khadyan Sahayta Yojna, Chhattisgarh:
   - This scheme provides 35 Kg rice at Rs.1/kg to 7.66 Lacs State AAY families; 10 kg rice free of cost to 60,277 destitute ration card holders and 10 kg rice at Rs. 1/kg to 8,194 disabled persons of the state every month.

2. Sanjay Gandhi Niradhar Anudan Yojana:
   - This scheme is applicable to destitute persons of age below 65 years, orphan children, all types of handicapped, person suffering from critical illness like T. B., cancer, AIDS, and leprosy, destitute widows (including those of farmers who committed suicide), destitute divorced women and women in the process of divorce, women freed from prostitution, and outraged women.

For more detailed information or to apply, you can visit the official [Government Department/Agency] website: https://www.india.gov.in/my-gover

In [37]:
import streamlit as st

In [45]:
# if 'messages' not in st.session_state:
#     st.session_state.messages=[
#         {
#             "role":"assistant",
#             "content":"Ask me anything"
#         }
#     ]
# for message in st.session_state.messages:
#     with st.chat_message(message["role"]):
#         st.markdown(message["content"])
def llm_function(query):
    query_result = index.search_doc(query)
    context = ""
    for i in range(5):
        for key, value in query_result[i].items():
            if(value>0.23):
                context += key + " "
    if context == '':
        print('Is there anything else I can help you with?')
        return 
    else:
        formatted_response = f''' Give response in this format, dont change the format or the links given-: 
        Hello {user_name},

        Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

        1. Scheme Name 1:
        - Information like eligibility, benefit.

        2. Scheme Name 2:
        - Information like eligibility, benefit.

        3. Scheme Name 3:
        - Information like eligibility, benefit.

        4. Scheme Name 4:
        - Information like eligibility, benefit.

        5. Scheme Name 5:
        - Information like eligibility, benefit.


        For more detailed information or to apply, you can visit the official [Government Department/Agency] website: {website}.


        If you have any more questions or need further assistance, feel free to ask!

        Best regards,
        GovSevak.

        *This message is generated by GENAI, GENAI can make mistakes. Consider checking important information.*

        QUERY:${query}$ CONTEXT:${context}$'''

        response = model.generate_content(formatted_response)
        print(response.text)
        with st.chat_message('user'):
            st.markdown(query)
        with st.chat_message('assistant'):
            st.markdown(response.text)
        st.session_state.messages.append(
            {
                'role':'user',
                'content':Query
            }
        )
        st.session_state.messages.append(
            {
                'role':'assitant',
                'content':response.text
            }
        )
    
if query:
    llm_function(query)
    

2024-03-08 17:09:58.693 
  command:

    streamlit run c:\Users\hp\Desktop\NLP-Project\env\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


Hello Kishan,

Thank you for reaching out! 🌟 Based on the information you provided, here are some government schemes that might be relevant to your situation:

1. Village Grain Bank Scheme by Department of Food and Public Distribution:
- Eligibility: Marginalized food insecure households
- Benefit: Provides safeguard against starvation during natural calamities or lean seasons

2. Mukhamantri Khadyan Sahayta Yojna, Chhattisgarh:
- Eligibility: AAY families, destitute ration card holders, disabled persons
- Benefit: Rice at subsidized rates or free of cost

3. Sanjay Gandhi Niradhar Anudan Yojana:
- Eligibility: Destitute persons below 65 years, orphans, handicapped, critically ill, destitute widows, divorced women, women freed from prostitution, outraged women
- Benefit: Financial assistance

For more detailed information or to apply, you can visit the official [Government Department/Agency] website: https://www.india.gov.in/my-government/schemes.


If you have any more questions or ne

AttributeError: st.session_state has no attribute "messages". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization

In [ ]:
import streamlit as st
import os
import google.generativeai as genai
genai.configure(api_key=os.environ.get('GOOGLE_API_KEY'))
model = genai.GenerativeModel('gemini-pro')
#initiate the chathistory
if 'messages' not in st.session_state:
    st.session_state.messages=[
        {
            "role":"assistant",
            "content":"Ask me anything"
        }
    ]
# Displying chathistory on restart.
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])
# Process, display and store query and response.
def llm_function(Query):
    response = model.generate_content(Query)
    # Displaying users message and assistant response as well.
    with st.chat_message('user'):
        st.markdown(Query)
    with st.chat_message('assistant'):
        st.markdown(response.text)
    # Storing user message and query
    st.session_state.messages.append(
        {
            'role':'user',
            'content':Query
        }
    )
    st.session_state.messages.append(
        {
            'role':'assitant',
            'content':response.text
        }
    )
query = st.chat_input("hey hey what's up big boi")
if query:
    llm_function(query)